In [2]:
#######################
# Elastic Net Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data/MixedModels'
DataBaseDir2 = '../../data/MixedModels2'
InputDir = '%s/l1/kfold' % DataBaseDir
InputDir2 = '%s/l1/kfold' % DataBaseDir2
OutputDir = '%s/l2/kfold' % DataBaseDir
OutputDir2 = '%s/l2/kfold' % DataBaseDir2
kfold = 5
TreeNum_30 = 100
TreeNum_60 = 100
InputCols_30 = ['lgb_l2_%s' % i for i in range(TreeNum_30)]
InputCols_60 = ['lgb_l2_%s' % i for i in range(TreeNum_60)]
SingleModelInputCols = ['lgb_l2', 'xgb_rmse','lgb_huber', 'lgb_fair', 
                        'etr', 'en', 'rf',  'gbr_ls', 'gbr_huber', 'gbr_lad', 'rgf_sib', 'lassolars', 'rgf',
                        'knn', 'knn_2', 'knn_4', 'knn_8', 'knn_16', 'knn_32']
strategy = 'lassolars'
start_time = datetime.datetime.now()
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
join_keys = ['air_store_id', 'visit_date']
start = time.time()
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    FoldInputDir2 = '%s/%s' % (InputDir2, fold)
    for i in range(len(InputCols_30)):
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, InputCols_30[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, InputCols_30[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, InputCols_30[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        if(i == 0):
            FoldValid = valid
            FoldHoldout = holdout
            FoldTest = test
        else:
            FoldValid['%s_30' % InputCols_30[i]] = valid[InputCols_30[i]]
            FoldHoldout['%s_30' % InputCols_30[i]] = holdout[InputCols_30[i]]
            FoldTest['%s_30' % InputCols_30[i]] = test[InputCols_30[i]]
    for i in range(len(InputCols_60)):
        valid2 = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir2, InputCols_60[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout2 = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir2, InputCols_60[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        test2 = pd.read_csv('%s/test_%s.csv' % (FoldInputDir2, InputCols_60[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        FoldValid['%s_60' % InputCols_60[i]] = valid2[InputCols_60[i]]
        FoldHoldout['%s_60' % InputCols_60[i]] = holdout2[InputCols_60[i]]
        FoldTest['%s_60' % InputCols_60[i]] = test2[InputCols_60[i]]
    FoldInputDir = '../../data/l1/kfold/%s' % (fold)
    for i in range(len(SingleModelInputCols)):
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, SingleModelInputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, SingleModelInputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, SingleModelInputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        FoldValid[SingleModelInputCols[i]] = valid[SingleModelInputCols[i]]
        FoldHoldout[SingleModelInputCols[i]] = holdout[SingleModelInputCols[i]]
        FoldTest[SingleModelInputCols[i]] = test[SingleModelInputCols[i]]
    ###
    FoldValid['fold'] = fold
    valid_dfs.append(FoldValid)
    holdout_dfs.append(FoldHoldout)
    test_dfs.append(FoldTest)
    end = time.time()
    print('fold %s done, time elapsed %ss' % (fold, (end - start)))
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
print('----------------- CHECK -------------')
print('\n============================ TrainData')
print(TrainData.head(5))
print('\n============================ HoldoutData')
print(holdout_dfs[0].head(5))
print('\n============================ TestData')
print(test_dfs[0].head(5))
print('-------------------------------------')
# sys.exit(1)
##### model selection with CV
# score
cv_score = .0
holdout_score = .0
# predict
y_test_pred = 0
for fold in range(kfold):
    FoldData = {
        'train': TrainData[TrainData['fold'] != fold],
        'valid': TrainData[TrainData['fold'] == fold],
        'holdout': holdout_dfs[fold],
        'test': test_dfs[fold]
    }
    col = [c for c in FoldData['train'].columns if c not in ['id', 'air_store_id', 'visit_date', 'visitors', 'hpg_store_id', 'fold']]
    # train
    model = linear_model.LassoLars(alpha= 0.0000001, max_iter= 100, verbose= True)
    model.fit(FoldData['train'][col].astype(np.float32, copy=False), FoldData['train']['visitors'].values.astype(np.float32, copy=False))
    # for valid
    FoldData['valid'][strategy] = model.predict(FoldData['valid'][col])
    rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
    cv_score += rmsle_valid
    # for holdout
    FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][col])
    rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
    holdout_score += rmsle_holdout
    # for test
    FoldData['test'][strategy] = model.predict(FoldData['test'][col])
    y_test_pred += FoldData['test'][strategy]

    print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
    #### output
    FoldOutputDir = '%s/%s' % (OutputDir, fold)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    for mod in FoldData.keys():
        if(mod == 'train'):
            continue
        OutCols = []
        if(mod == 'test'):
            OutCols.append('id')
        OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
        OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
        OutFoldData = FoldData[mod][OutCols]
        OutFoldData.to_csv(OutputFile, index= False)
    print('saving for %sth fold data done.' % (fold))
    
y_test_pred /= kfold  # Average test set predictions
cv_score /= kfold # Average valid set predictions
holdout_score /= kfold # Average holdout set predictions

# Create submission file
sub = pd.DataFrame()
sub['id'] = test_dfs[0]['id']
sub['visitors'] = np.expm1(y_test_pred)
OutputFileName = '%s_submit_%s' % (strategy, datetime.datetime.now().strftime("%Y-%m-%d"))
SubmitDir = '%s/l2/submit' % DataBaseDir
if(os.path.exists(SubmitDir) == False):
    os.makedirs(SubmitDir)
sub.to_csv('%s/%s.csv' % (SubmitDir, OutputFileName), float_format='%.6f', index=False)
print('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))
os.system('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))

finish_time = datetime.datetime.now()
elapsed = (finish_time - start_time).seconds
print('\n======================')
print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, elapsed))
print('======================\n')

fold 0 done, time elapsed 35.893953800201416s
fold 1 done, time elapsed 74.9752037525177s
fold 2 done, time elapsed 113.19851589202881s
fold 3 done, time elapsed 151.7770550251007s
fold 4 done, time elapsed 191.95786595344543s
----------------- CHECK -------------

============================ TrainData
           air_store_id visit_date  visitors  lgb_l2_0  lgb_l2_1_30  \
0  air_ba937bf13d40fb24 2016-01-15  3.401197  3.521467     3.136138   
1  air_ba937bf13d40fb24 2016-01-29  3.295837  3.543300     3.228056   
2  air_ba937bf13d40fb24 2016-01-30  1.945910  3.357661     3.333841   
3  air_ba937bf13d40fb24 2016-02-10  3.496508  3.229475     3.235015   
4  air_ba937bf13d40fb24 2016-02-13  2.197225  3.356503     3.297490   

   lgb_l2_2_30  lgb_l2_3_30  lgb_l2_4_30  lgb_l2_5_30  lgb_l2_6_30  ...   \
0     3.554459     3.394433     3.562601     3.264874     3.488428  ...    
1     3.592054     3.394433     3.562601     3.293011     3.488428  ...    
2     3.406665     3.325825     3.393871

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fold 0: valid score 0.489057, holdout score 0.480393, valid length 45371
saving for 0th fold data done.
fold 1: valid score 0.479824, holdout score 0.480662, valid length 45371
saving for 1th fold data done.
fold 2: valid score 0.482424, holdout score 0.479600, valid length 45371
saving for 2th fold data done.
fold 3: valid score 0.486018, holdout score 0.479532, valid length 45370
saving for 3th fold data done.
fold 4: valid score 0.480853, holdout score 0.479878, valid length 45370
saving for 4th fold data done.
zip ../../data/MixedModels/l2/submit/lassolars_submit_2018-01-16.zip ../../data/MixedModels/l2/submit/lassolars_submit_2018-01-16.csv

CV score 0.4836, Holdout score 0.4800, Elapsed time: 209.00s

